# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense
import pprint

In [2]:
songs = pd.read_csv("song_list5.csv")

In [3]:
songs.head()

,songid,artist,track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5X4Qm0rVLcZeeO4tSDmBg3,Jack Bruce,Running Thro' Our Hands,0.456,0.255,9.0,-15.805,1.0,0.0480,0.94600,0.170000,0.951,0.0532,116.424,253067.0,4.0
1,1pNpt53PZPet9dvJN3RKGr,Prefuse 73,Parachute Panador,0.535,0.806,7.0,-10.289,1.0,0.0642,0.00436,0.019100,0.457,0.3760,90.089,63733.0,4.0
2,3oxz2oCzAWdPzA6In2zA5u,Pasion Vega,La Gata Bajo La Lluvia,0.294,0.482,5.0,-6.406,1.0,0.0430,0.46300,0.000000,0.335,0.2040,166.693,255280.0,3.0
3,05JGVUwt7XJk5FPqH0Wsch,Jonny Lang,Walking Away,0.563,0.631,0.0,-5.144,1.0,0.0324,0.06350,0.000008,0.163,0.5400,115.657,254827.0,4.0
4,3xdgCFMTn6ut8fZYxfAuR0,Skye,All the Promises,0.358,0.611,2.0,-9.752,0.0,0.0454,0.51500,0.000468,0.149,0.1640,171.596,256933.0,4.0


In [4]:
songs.dtypes

songid               object
artist               object
track                object
danceability        float64
energy              float64
key                 float64
loudness            float64
mode                float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms         float64
time_signature      float64
dtype: object

In [5]:
features = songs[[
    "danceability", "energy", "key", "loudness", "mode", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", "tempo", 
    "duration_ms", "time_signature"]].to_numpy()

In [6]:
features[0]

array([ 4.56000e-01,  2.55000e-01,  9.00000e+00, -1.58050e+01,
        1.00000e+00,  4.80000e-02,  9.46000e-01,  1.70000e-01,
        9.51000e-01,  5.32000e-02,  1.16424e+02,  2.53067e+05,
        4.00000e+00])

In [7]:
scaler = StandardScaler()

In [8]:
x_train = scaler.fit_transform(features)
# x_train = features

In [9]:
print(x_train[0])
print(x_train[0].shape)

[-0.47695143 -1.43616823  1.03467011 -1.32564479  0.70711739 -0.34028656
  1.91669133 -0.14321669  3.87500921 -1.70714343 -0.17066299  0.03404049
  0.24368604]
(13,)


In [10]:
# encoder_input = Input(shape=(13,))

# encoder_dense1 = Dense(10, activation='relu')(encoder_input)
# encoder_dense2 = Dense(8, activation='relu')(encoder_dense1)
# encoder_dense3 = Dense(6, activation='relu')(encoder_dense2)
# encoder_output = Dense(4, activation='relu')(encoder_dense3)

# encoder = keras.Model(encoder_input, encoder_output)



# decoder_dense1 = Dense(6, activation='relu')(encoder_output)
# decoder_dense2 = Dense(8, activation='relu')(decoder_dense1)
# decoder_dense3 = Dense(10, activation='relu')(decoder_dense2)
# decoder_output = Dense(13)(decoder_dense3)

# auto_encoder = keras.Model(encoder_input, decoder_output)

# auto_encoder.summary()

In [11]:
# Model Version 2

encoder_input = Input(shape=(13,))

encoder_dense1 = Dense(12, activation='relu')(encoder_input)
encoder_dense2 = Dense(10, activation='relu')(encoder_dense1)
encoder_dense3 = Dense(8, activation='relu')(encoder_dense2)
encoder_dense4 = Dense(6, activation='relu')(encoder_dense3)
encoder_output = Dense(2, activation='relu')(encoder_dense4)

encoder = keras.Model(encoder_input, encoder_output)

decoder_dense2 = Dense(6, activation='relu')(encoder_output)
decoder_dense3 = Dense(8, activation='relu')(decoder_dense2)
decoder_dense4 = Dense(10, activation='relu')(decoder_dense3)
decoder_dense5 = Dense(12, activation='relu')(decoder_dense4)
decoder_output = Dense(13)(decoder_dense5)
# decoder_output = Dense(13, activation='softplus')(decoder_dense5)

auto_encoder = keras.Model(encoder_input, decoder_output)

auto_encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 12)                168       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                130       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 14        
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 18  

In [12]:
auto_encoder.compile(optimizer='nadam',
                    loss='mean_absolute_error',
                    metrics='mean_absolute_percentage_error')

In [13]:
auto_encoder.fit(x_train, x_train,
                epochs=100,
                batch_size=32,
                shuffle=True)

Epoch 1/100
1563/1563 [==============================] - 3s 2ms/step - loss: 0.5882 - mean_absolute_percentage_error: 179.9081
Epoch 2/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.5080 - mean_absolute_percentage_error: 184.2152
Epoch 3/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.4926 - mean_absolute_percentage_error: 185.0478
Epoch 4/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.4808 - mean_absolute_percentage_error: 191.8038
Epoch 5/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.4722 - mean_absolute_percentage_error: 186.1577
Epoch 6/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.4672 - mean_absolute_percentage_error: 186.0746
Epoch 7/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.4649 - mean_absolute_percentage_error: 187.4635
Epoch 8/100
1563/1563 [==============================] - 2s 1ms/step - loss: 0.4633 - mean_absolute_percentage_

In [14]:
preds = auto_encoder.predict(x_train)

In [15]:
pprint.pprint(preds[0])
pprint.pprint(x_train[0])
print("============================================================")
pprint.pprint(preds[1])
pprint.pprint(x_train[1])
print("============================================================")
pprint.pprint(preds[2])
pprint.pprint(x_train[2])

array([-0.8798475 , -1.7275432 , -0.18597168, -1.1995802 ,  0.70768386,
       -0.41527253,  1.6717181 , -0.5876498 , -0.47801736, -1.6586576 ,
       -0.55115545, -0.33200756,  0.24512994], dtype=float32)
array([-0.47695143, -1.43616823,  1.03467011, -1.32564479,  0.70711739,
       -0.34028656,  1.91669133, -0.14321669,  3.87500921, -1.70714343,
       -0.17066299,  0.03404049,  0.24368604])
array([ 0.44521952, -0.2880424 , -0.1336196 , -0.06228232,  0.7082029 ,
       -0.31183374, -0.7158272 , -0.6356993 , -0.4974471 , -0.49354   ,
       -0.26810828,  0.03757627,  0.24511549], dtype=float32)
array([-0.03739907,  0.69843612,  0.47428756, -0.17173186,  0.70711739,
       -0.1963104 , -0.87635746, -0.58458172,  1.27879637, -0.49117162,
       -1.06220066, -1.51723037,  0.24368604])
array([-0.45992905, -0.64994824, -0.1795414 ,  0.68798745,  0.707829  ,
       -0.4516239 ,  0.9742821 , -0.63741523, -0.43464142, -1.1445893 ,
       -0.35879743, -0.14266393,  0.24527666], dtype=float32)


In [19]:
encoder.save('encoder')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: encoder\assets


In [17]:
# # auto_encoder.save("/nnmodels/auto_encoder")

# filepath = "/nnmodels/auto_encoder"

# tf.keras.models.save_model(
#     auto_encoder, filepath, overwrite=True, include_optimizer=True, save_format=None,
#     signatures=None, options=None
# )

In [18]:
# from tensorflow import keras
# model = keras.models.load_model('path/to/location')